For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __YOUR NAMES HERE__

Step 1: Train your own word embeddings
--------------------------------

(describe the provided dataset that you have chosen here)

Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties.

(describe your dataset here)

In [44]:
# import your libraries here
import nltk
import keras
import numpy as np
from gensim.models import KeyedVectors

### a) Train embeddings on GIVEN dataset

In [ ]:
# code to train your word embeddings

# Read the file and prepare the training data 
# so that it is in the following format

# data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
# 			['this', 'is', 'the', 'second', 'sentence'],
# 			['yet', 'another', 'sentence'],
# 			['one', 'more', 'sentence'],
# 			['and', 'the', 'final', 'sentence']]


In [27]:
from gensim.models import Word2Vec

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# vector_size = EMBEDDINGS_SIZE
# min_count = 1

In [ ]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size {}'.format(len(model.wv.vocab)))

In [ ]:
# You can save file in txt format, then load later if you wish.
# model.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [ ]:
# then do a second data set

What text-normalization and pre-processing did you do and why? __YOUR ANSWER HERE__

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

## Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [51]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

# The size of the ngram language model you want to train
# change as needed for your experiments
NGRAM = 3 

# Initializing a Tokenizer
# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
with open("based.txt","r",encoding="utf-8") as f:
    lines = [l for l in f.readlines() if len(l.strip()) > 0]
corpus = lines
# corpus = ["Hello world.","Goodbye cruel world.","Hello, love.","This world is the best world, because you are in it."]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
# encoded = tokenizer.texts_to_sequences(["Hello, my love, my world.","I love you more than the world itself."])
encoded = tokenizer.texts_to_sequences(["We live in a society.", "Children yearn for the mines!"])
print(encoded)
len(tokenizer.word_counts)

[[24, 287, 8, 4, 20], [204, 11, 1]]


4126

### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

In [24]:
def generate_ngram_training_samples(encoded: list[list[int]], ngram:int) -> list[list[int]]:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    # flattened list comprehension
    samples = [sentence[i:i+ngram] for sentence in encoded for i in range(len(sentence)-ngram+1)]
    return samples

ngram_samples = generate_ngram_training_samples(encoded, NGRAM)
print("ngram_samples:",ngram_samples)
# [f"{a},{b}" for b in range(5) for a in range(b)]


ngram_samples: [[24, 287, 8], [287, 8, 4], [8, 4, 20], [204, 11, 1]]


### c) Then, split the sequences into X and y and create a Data Generator

In [65]:
# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]

# returns X and y using given encoded ngram training samples
def split_X_y(samples: list[list[int]]) -> tuple[list[list[int]],list[int]]:
    return np.array([l[:-1] for l in samples]), np.array([l[-1] for l in samples])

X, y = split_X_y(ngram_samples)
print("X:",X)
print("y:",y)

X: [[ 24 287]
 [287   8]
 [  8   4]
 [204  11]]
y: [ 8  4 20  1]


In [66]:
# JUST FOR TESTING PURPOSES SINCE I DONT HAVE WORD EMBEDDINGS TO WORK WITH
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [76]:
def read_embeddings(tokenizer: Tokenizer, model: KeyedVectors) -> tuple[dict[str,list], dict[int,list]]:
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    word_embeddings = {w:model[w] for w in tokenizer.word_index if w in model}
    encoded_embeddings = {tokenizer.word_index[w]:word_embeddings[w] for w in word_embeddings}
    return word_embeddings, encoded_embeddings


word_embed, enc_embed = read_embeddings(tokenizer,glove_vectors)

25

In [80]:
def data_generator(X: list, y: list, batch_size: int,tokenizer: Tokenizer, enc_embed:dict) -> tuple[list,list]:
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    i = 0
    for i in range(0,len(y),batch_size):
        # for each sequence in the batch, flatten all word embedding vectors into one vector
        embeddings = np.array([[weight for word_index in sequence for weight in enc_embed[word_index]] for sequence in X[i:i+batch_size]])
        labels = to_categorical(y[i:i+batch_size],num_classes=len(tokenizer.word_counts))
        yield embeddings, labels
    pass



In [81]:
# Examples
# initialize data_generator
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch = len(ngram_samples)//num_sequences_per_batch  # Number of batches per epoch
train_generator = data_generator(X, y, num_sequences_per_batch, tokenizer, enc_embed)

sample=next(train_generator) # this is how you get data out of generators
print(sample[0].shape) # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
print(sample[1].shape)   # (batch_size, |V|) to_categorical

(4, 50)
(4, 4126)


### d) Train your models

In [70]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models

# Define the model architecture using Keras Sequential API



In [71]:
# Start training the model
model.fit(x=train_generator, 
          steps_per_epoch=steps_per_epoch,
          epochs=1)

NameError: name 'model' is not defined

### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

You may find it useful to run your HW 2 code on one of the datasets (or a subset of the dataset) that you used for this homework.

Sources Cited
----------------------------
